In [1]:
import os
import errno  
import urllib.request, re, requests, urllib.parse
import time
import shutil
from PIL import Image

urlopenheader={ 'User-Agent' : 'Mozilla/5.0 (X11; Fedora; Linux x86_64; rv:42.0) Gecko/20100101 Firefox/42.0',
                "QProxy-Token": "5eXfF1"}
print("hello")

hello


In [2]:
def mkdir_p(path):
    try:
        os.makedirs(path)
    except OSError as exc:  # Python >2.5
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise

In [3]:
def getHtml(request_url):
#     request=urllib.request.Request(request_url,None,headers=urlopenheader)
#     response=urllib.request.urlopen(request)
    page = requests.get(request_url)
    html_contents = page.text
    html = html_contents
    time.sleep(0.1)
#     print(type(html))
#     html = response.read().decode('utf8')
    return html

In [4]:
def download(download_info):
    (url, path) = download_info
    for i in range(3):
        time.sleep(0.1)
#         print("trying %d times." % i)
        try:
            r = requests.get(url, timeout=1, stream=True)
            if r.status_code == 200:
                with open(path, 'wb') as f:
                    r.raw.decode_content = True
                    shutil.copyfileobj(r.raw, f) 
                    return 1
            
        except:
            time.sleep(1)
            pass
    return 0
# def mass_download(urls, nthread):
#     print('Downloading...')
#     download_infos = [(url, os.path.basename(url)) for url in urls]
#     with Pool(nthread) as pool:
#         pool.map(download, download_infos)        

In [5]:

def getSGImagesKW(keyword, imageDir, imageNum=100):
    print(keyword)
    start = 1
    downloadedNum = 0
    mkdir_p(imageDir)
    file_pre=re.split(r"\\|/", imageDir)[-1]+"_"
    while downloadedNum<imageNum:
        print("download %s images." % downloadedNum)
        curDLNum=getSGImagesSingle(keyword, imageDir,start, file_pre)
        downloadedNum += curDLNum
        start=downloadedNum+1


In [6]:

def getSGImagesSingle(keyword, imageDir, start=1, file_pre="dumpling_0424_"):
#     print(urllib.parse.quote(keyword))
#     url = "http://pic.sogou.com/pics?ie=utf8&mode=1&start=%d&reqType=ajax&reqFrom=result&tn=0&query=%s&" % (start, urllib.parse.quote(keyword))    
    url = "http://images.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word=%s&pn=%d&gsm=&" % (urllib.parse.quote(keyword), start)    
    ind=start
    webcontent = getHtml(url)
    imageUrlSet=set([])
    pic_url = re.findall('"objURL":"([^"]*?)"',webcontent)  
    format_set = set(["jpg", "jpeg", "png"])
#     imageUrlSet.update(pic_url)
    for i in range(len(pic_url)):
#         print(i+1, pic_url[i])
        fpath = imageDir+"/"+os.path.basename(pic_url[i])
        res=download((pic_url[i], fpath))
        if res==1:
            imageUrlSet.add(pic_url[i])
            im=None
            try:
                if  os.path.exists(fpath) and os.path.isfile(fpath):
                    fsize = os.path.getsize(fpath)
                    if fsize<=20*1024:
                        os.remove(fpath)
                        continue
                im = Image.open(fpath)
                realFormat=im.format.lower()
                imgSize = im.size
                maxSize = max(imgSize)
                minSize = min(imgSize)
                im.close()
                nameFormat=fpath.split(".")[-1].lower()
#                 print(fpath, maxSize)
                if realFormat in format_set and maxSize>=300 and minSize>=300:
                    fpath_2 = imageDir + "/" + file_pre + ("0424_%d." % ind) + realFormat
#                     print(fpath_2)
                    os.rename(fpath, fpath_2)
                else:
                    if im!=None:
                        im.close()
                    os.remove(fpath)
                
            except:
                if im!=None:
                    im.close()
#                 os.remove(fpath)
            ind += 1
                    
    return len(pic_url)
#         saveImage(i+1, decodeHtml(pic_url[i]))


In [7]:
def getList(url="https://www.taobao.com/markets/tbhome/market-list", reg="""<a class="category-name".*?>([^>]+)</a>"""):
    webcontent = getHtml(url)
    eles = re.findall(r"%s" % reg,webcontent)
    return eles
    

In [8]:
import os, sys, pinyin

def getFiles(dir1):
    if os.path.exists(dir1) and os.path.isdir(dir1):
        print("%s is dir." % dir1)
        return os.listdir(dir1)

def renameFileToIndex(dir1, file_pre="dumpling_"):
    if os.path.exists(dir1) and os.path.isdir(dir1):
        print("%s is dir." % dir1)
        count=0
        for f1 in os.listdir(dir1):
            f2=dir1+"/"+f1
            if os.path.isfile(f2):
                isuf=f1.rfind(".")
                f3=dir1+"/"+file_pre+ str(count)+f1[isuf:]
                os.rename(f2, f3)
                count += 1

In [9]:

def findWrongFormatImage(dir1):
    needToDel=set([])
    format_set = set(["jpg", "jpeg", "png"])
#     imageUrlSet.update(pic_url)
    file_pre=re.split(r"\\|/", dir1)[-1]+"_0424_"
    start = 0
    for d1 in os.listdir(dir1):
        fpath=dir1+"/"+d1
#         print(fpath)
        im=None
        try:
            if  os.path.exists(fpath) and os.path.isfile(fpath):
                fsize = os.path.getsize(fpath)
                if fsize<=20*1024:
                    os.remove(fpath)
                    continue
            im = Image.open(fpath)
            realFormat=im.format.lower()
            imgSize = im.size
            maxSize = max(imgSize)
            minSize = min(imgSize)
            im.close()
            nameFormat=fpath.split(".")[-1].lower()
#             print(realFormat, maxSize, minSize)
            if realFormat in format_set and maxSize>=300 and minSize>=300:
                fpath_2 = dir1 + "/" + file_pre + ("%d." % start) + realFormat
                os.rename(fpath, fpath_2)
            else:
                if im!=None:
                    im.close()
                os.remove(fpath)

        except:
#             raise
#             print("except")
            if im!=None:
                im.close()
            os.remove(fpath)
        start += 1

In [10]:
#"binggan dianxin haizhe jianbingguozi jutiao niangao pidong shanyao suanban tudouni wanyue xiaochuan"
for p in "xiaodangao yinyuanbao yuyuan ".split():
    findWrongFormatImage(r"E:\data\image\baidu20170424\%s" % p)

In [23]:
renameFileToIndex(r"E:\data\image\baidu20170502\lotus", file_pre="lotus_0502_")

E:\data\image\baidu20170502\lotus is dir.


In [ ]:
renameFileToIndex(r"E:\data\image\hongmaojiaozi", file_pre="dumpling_0427_")

In [ ]:
tbGoods=getList()

In [ ]:
print(tbGoods)

In [ ]:
s1="map3/*:\?\"\'<>|mp4"
s1_2=re.sub(r"""/|\\|\*|\:|\?|\"|\<|\>|\||\'""", "", s1 )
print(s1_2)

In [ ]:
for ele in tbGoods:
    qw=ele
    dir1="E:\\data\\image\\baidu\\%s" % re.sub(r"""/|\\|\*|\:|\?|\"|\<|\>|\||\'""", "", qw )
    print(qw)
    print(dir1)
    getSGImagesKW(qw, dir1, imageNum=30)

In [ ]:
getSGImagesKW("红色台灯", "E:\\data\\image\\baidu\\红色台灯", imageNum=300)

In [ ]:
getSGImagesKW("小蜡烛", "E:\\data\\image\\baidu\\小蜡烛", imageNum=100)

In [ ]:
getSGImagesKW("圆形小蜡烛", "E:\\data\\image\\baidu\\圆形小蜡烛", imageNum=30)

getSGImagesKW("粗蜡烛", "E:\\data\\image\\baidu\\粗蜡烛", imageNum=30)

In [ ]:
print("铁皮盒 圆形")
getSGImagesKW("铁皮盒 圆形", "E:\\data\\image\\baidu\\圆形铁皮盒", imageNum=220)
print("纸质 红包")
getSGImagesKW("纸质 红包", "E:\\data\\image\\baidu\\纸质红包", imageNum=220)
print("花灯")
getSGImagesKW("花灯", "E:\\data\\image\\baidu\\花灯", imageNum=220)
print("红色旅行箱")
getSGImagesKW("红色旅行箱", "E:\\data\\image\\baidu\\红色旅行箱", imageNum=220)

In [ ]:
getSGImagesKW("彩色装饰球", "E:\\data\\image\\baidu\\彩色装饰球", imageNum=300)

In [ ]:
getSGImagesKW("装饰球", "E:\\data\\image\\baidu\\装饰球", imageNum=300)

In [ ]:
getSGImagesKW("圆柱形 红色", "E:\\data\\image\\baidu\\圆柱形红色", imageNum=300)

In [ ]:
getSGImagesKW("电线", "E:\\data\\image\\baidu\\dianxian", imageNum=300)

In [ ]:
getSGImagesKW("徽章", "E:\\data\\image\\baidu20170418\\huizhang", imageNum=300)
getSGImagesKW("拖鞋", "E:\\data\\image\\baidu20170418\\tuoxie", imageNum=300)
getSGImagesKW("线圈", "E:\\data\\image\\baidu20170418\\xianquan", imageNum=300)
getSGImagesKW("毛线", "E:\\data\\image\\baidu20170418\\maoxian", imageNum=300)
getSGImagesKW("垃圾袋", "E:\\data\\image\\baidu20170418\\lajidai", imageNum=300)
getSGImagesKW("沐浴球", "E:\\data\\image\\baidu20170418\\muyuqiu", imageNum=300)
getSGImagesKW("风筝", "E:\\data\\image\\baidu20170418\\fengzheng", imageNum=300)


In [ ]:
getSGImagesKW("国徽", "E:\\data\\image\\baidu20170418\\guohui", imageNum=300)
getSGImagesKW("红色 徽章", "E:\\data\\image\\baidu20170418\\hongsehuizhang", imageNum=300)

In [ ]:
getSGImagesKW("荷包", "E:\\data\\image\\baidu20170418\\hebao", imageNum=300)
getSGImagesKW("传统 绣球", "E:\\data\\image\\baidu20170418\\chuantongxiuqiu", imageNum=300)

In [ ]:
getSGImagesKW("拿气球", "E:\\data\\image\\baidu20170419\\qiqiu", imageNum=300)
getSGImagesKW("红色 爱心", "E:\\data\\image\\baidu20170419\\hongseaixin", imageNum=300)

In [ ]:
getSGImagesKW("红色 心形 气球", "E:\\data\\image\\baidu20170419\\hongsexinxingqiqiu", imageNum=300)

In [ ]:
getSGImagesKW("蒸笼", "E:\\data\\image\\baidu20170419\\zhuzhenglong", imageNum=300)

In [ ]:
getSGImagesKW("鼓", "E:\\data\\image\\baidu20170419\\gu", imageNum=500)

In [ ]:
getSGImagesKW("饺子", "E:\\data\\image\\baidu20170419\\gu", imageNum=500)

In [ ]:
#"馒头， 元宵， 花卷， 小蛋糕， 元宝， 混沌， 锯条， 包子， 点心， 年糕， 煎饼果子， 蒜瓣， 弯月， 小船"

getSGImagesKW("混沌", "E:\\data\\image\\baidu20170424\\hundun", imageNum=500)
getSGImagesKW("馒头", "E:\\data\\image\\baidu20170424\\mantou", imageNum=500)
getSGImagesKW("元宵", "E:\\data\\image\\baidu20170424\\yuanxiao", imageNum=500)
getSGImagesKW("花卷", "E:\\data\\image\\baidu20170424\\huajuan", imageNum=500)
getSGImagesKW("小蛋糕", "E:\\data\\image\\baidu20170424\\xiaodangao", imageNum=500)
getSGImagesKW("元宝", "E:\\data\\image\\baidu20170424\\yuanbao", imageNum=500)
getSGImagesKW("锯条", "E:\\data\\image\\baidu20170424\\jutiao", imageNum=500)
getSGImagesKW("包子", "E:\\data\\image\\baidu20170424\\baozi", imageNum=500)
getSGImagesKW("点心", "E:\\data\\image\\baidu20170424\\dianxin", imageNum=500)
getSGImagesKW("年糕", "E:\\data\\image\\baidu20170424\\niangao", imageNum=500)
getSGImagesKW("煎饼果子", "E:\\data\\image\\baidu20170424\\jianbingguozi", imageNum=500)
getSGImagesKW("蒜瓣", "E:\\data\\image\\baidu20170424\\suanban", imageNum=500)

In [ ]:
#"馄钝, 银元宝， 汤圆， 烧麦"
#"馄钝, 银元宝， 汤圆， 烧麦"
getSGImagesKW("弯月", "E:\\data\\image\\baidu20170424\\wanyue", imageNum=500)
getSGImagesKW("小船", "E:\\data\\image\\baidu20170424\\xiaochuan", imageNum=500)
getSGImagesKW("馄钝", "E:\\data\\image\\baidu20170424\\hundun2", imageNum=500)
getSGImagesKW("银元宝", "E:\\data\\image\\baidu20170424\\yinyuanbao", imageNum=500)
getSGImagesKW("汤圆", "E:\\data\\image\\baidu20170424\\tangyuan", imageNum=500)
getSGImagesKW("烧麦", "E:\\data\\image\\baidu20170424\\shaomai", imageNum=500)

In [ ]:
getSGImagesKW("饼干", "E:\\data\\image\\baidu20170424\\binggan", imageNum=500)
getSGImagesKW("肠粉", "E:\\data\\image\\baidu20170424\\changfen", imageNum=500)
getSGImagesKW("年糕", "E:\\data\\image\\baidu20170424\\niangao", imageNum=500)
getSGImagesKW("皮冻", "E:\\data\\image\\baidu20170424\\pidong", imageNum=500)
getSGImagesKW("海蜇", "E:\\data\\image\\baidu20170424\\haizhe", imageNum=500)
getSGImagesKW("土豆泥", "E:\\data\\image\\baidu20170424\\tudouni", imageNum=500)
getSGImagesKW("核桃仁", "E:\\data\\image\\baidu20170424\\hetaoren", imageNum=500)
getSGImagesKW("芋圆", "E:\\data\\image\\baidu20170424\\yuyuan", imageNum=500)

In [ ]:
getSGImagesKW("山药 去皮", "E:\\data\\image\\baidu20170424\\shanyao", imageNum=500)


In [ ]:
getSGImagesKW("柚子瓣", "E:\\data\\image\\baidu20170424\\youziban", imageNum=120)
getSGImagesKW("桔子瓣", "E:\\data\\image\\baidu20170424\\juziban", imageNum=120)
getSGImagesKW("藕合", "E:\\data\\image\\baidu20170424\\ouhe", imageNum=120)
getSGImagesKW("炸油饼", "E:\\data\\image\\baidu20170424\\zhayoubing", imageNum=120)
getSGImagesKW("土豆片", "E:\\data\\image\\baidu20170424\\tudoupian", imageNum=120)
getSGImagesKW("虾片", "E:\\data\\image\\baidu20170424\\xiapian", imageNum=120)
getSGImagesKW("炸鸡", "E:\\data\\image\\baidu20170424\\zhaji", imageNum=120)
getSGImagesKW("花生", "E:\\data\\image\\baidu20170424\\huasheng", imageNum=120)
getSGImagesKW("手背", "E:\\data\\image\\baidu20170424\\shoubei", imageNum=120)

In [ ]:

getSGImagesKW("白色结", "E:\\data\\image\\baidu20170424\\baisejie", imageNum=20)
getSGImagesKW("油条", "E:\\data\\image\\baidu20170424\\youtiao", imageNum=120)
getSGImagesKW("红薯块", "E:\\data\\image\\baidu20170424\\hongshukuai", imageNum=120)
getSGImagesKW("土豆块", "E:\\data\\image\\baidu20170424\\tudoukuai", imageNum=120)
getSGImagesKW("山药块", "E:\\data\\image\\baidu20170424\\shanyaokuai", imageNum=120)

In [ ]:
getSGImagesKW("花式饺子", "E:\\data\\image\\baidu20170427\\huashijiaozi", imageNum=300)
getSGImagesKW("韭菜盒子", "E:\\data\\image\\baidu20170427\\jiucaihezi", imageNum=300)

In [ ]:
getSGImagesKW("百合片", "E:\\data\\image\\baidu20170428\\baihepian", imageNum=500)
getSGImagesKW("苹果干", "E:\\data\\image\\baidu20170428\\pingguogan", imageNum=300)
getSGImagesKW("榴莲干", "E:\\data\\image\\baidu20170428\\liuliangan", imageNum=300)
getSGImagesKW("菠萝蜜干", "E:\\data\\image\\baidu20170428\\boluomigan", imageNum=300)

In [ ]:
getSGImagesKW("香蕉片", "E:\\data\\image\\baidu20170428\\xiangjiaogan", imageNum=200)

In [ ]:
getSGImagesKW("猫耳朵面", "E:\\data\\image\\baidu20170428\\maoerduomian", imageNum=200)

In [ ]:
getSGImagesKW("烟花", "E:\\data\\image\\baidu20170502\\yanhua", imageNum=500)




In [12]:
getSGImagesKW("手拿烟花", "E:\\data\\image\\baidu20170502\\shounayanhua", imageNum=300)


手拿烟花
download 0 images.


PermissionError: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'E:\\data\\image\\baidu20170502\\shounayanhua/10376187_600.jpg'

In [10]:
getSGImagesKW("圆气球", "E:\\data\\image\\baidu20170502\\yuanqiqiu", imageNum=500)
getSGImagesKW("气球拱门", "E:\\data\\image\\baidu20170502\\qiqiugongmen", imageNum=500)
getSGImagesKW("帐篷", "E:\\data\\image\\baidu20170502\\zhangpeng", imageNum=500)
getSGImagesKW("蒙古包", "E:\\data\\image\\baidu20170502\\menggubao", imageNum=500)

圆气球
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
气球拱门
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
帐篷
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
蒙古包
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.


In [11]:
getSGImagesKW("室内帐篷", "E:\\data\\image\\baidu20170502\\shineizhangpeng", imageNum=300)

室内帐篷
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.


In [12]:
getSGImagesKW("卡通氢气球", "E:\\data\\image\\baidu20170502\\katongqingqiqiu", imageNum=300)

卡通氢气球
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.


In [13]:
getSGImagesKW("买气球", "E:\\data\\image\\baidu20170502\\mai3qiqiu", imageNum=300)

买气球
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.


In [23]:
getSGImagesKW("蚊帐", "E:\\data\\image\\baidu20170502\\wenzhang4", imageNum=300)

蚊帐
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.


In [24]:
getSGImagesKW("枯树", "E:\\data\\image\\baidu20170502\\kushu", imageNum=300)
getSGImagesKW("冬天 枯树", "E:\\data\\image\\baidu20170502\\dongtiankushu", imageNum=300)
getSGImagesKW("红叶", "E:\\data\\image\\baidu20170502\\hongye", imageNum=500)
getSGImagesKW("黄叶", "E:\\data\\image\\baidu20170502\\huangye", imageNum=500)
getSGImagesKW("枫叶", "E:\\data\\image\\baidu20170502\\fengye", imageNum=500)
getSGImagesKW("梅花", "E:\\data\\image\\baidu20170502\\meihua", imageNum=500)
getSGImagesKW("樱花", "E:\\data\\image\\baidu20170502\\yinghua", imageNum=500)
getSGImagesKW("荷花", "E:\\data\\image\\baidu20170502\\hehua", imageNum=500)
getSGImagesKW("牡丹", "E:\\data\\image\\baidu20170502\\mudan", imageNum=500)
getSGImagesKW("迎春花", "E:\\data\\image\\baidu20170502\\yingchunhua", imageNum=500)
getSGImagesKW("桃花", "E:\\data\\image\\baidu20170502\\taohua", imageNum=500)
getSGImagesKW("月季", "E:\\data\\image\\baidu20170502\\yueji", imageNum=500)
getSGImagesKW("菊花", "E:\\data\\image\\baidu20170502\\juhua", imageNum=500)
getSGImagesKW("油菜花", "E:\\data\\image\\baidu20170502\\youcaihua", imageNum=500)
getSGImagesKW("薰衣草", "E:\\data\\image\\baidu20170502\\xunyicao", imageNum=500)
getSGImagesKW("玉兰花", "E:\\data\\image\\baidu20170502\\yulanhua", imageNum=500)
getSGImagesKW("桂花", "E:\\data\\image\\baidu20170502\\guihua", imageNum=500)




枯树
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
冬天 枯树
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
红叶
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
黄叶
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
枫叶
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
梅花
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
樱花
download 0 images.
download 

In [16]:

getSGImagesKW("牡丹花摄影高清图片", "E:\\data\\image\\baidu20170502\\mudan", imageNum=300)
getSGImagesKW("梅花摄影高清图片", "E:\\data\\image\\baidu20170502\\meihua", imageNum=500)

牡丹花摄影高清图片
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
梅花摄影高清图片
download 0 images.
download 60 images.
download 120 images.
download 180 images.
download 240 images.
download 300 images.
download 360 images.
download 420 images.
download 480 images.
